# Libraries & Settings

In [1]:
%load_ext autoreload
%autoreload

import pandas as pd
import numpy as np
import qgrid
#import beakerx as bx
from datetime import datetime as dtt

from pa_lib.file import data_files, load_bin, store_bin, store_excel
from pa_lib.data import (calc_col_partitioned, clean_up_categoricals, flatten, 
                         replace_col, cond_col, desc_col, unfactorize)
from pa_lib.util import obj_size, cap_words
from pa_lib.log  import time_log, info
from pa_lib.types import dtFactor
from pa_lib.vis import dive

# display long columns completely
pd.set_option('display.max_colwidth', 200)

pd.set_option('display.max_rows', 200)

# Load data

In [4]:
bd_raw = load_bin('bd_data.feather').rename(mapper=lambda name: cap_words(name, sep='_'), axis='columns')
bd = bd_raw.loc[(bd_raw.Netto >= 0)].pipe(clean_up_categoricals)

2019-07-11 14:01:16 [INFO] Reading from file C:\Users\kpf\data\bd_data.feather
2019-07-11 14:01:16 [INFO] Finished loading binary file in 0.14s (0.59s CPU)


In [3]:
desc_col(bd)

,DTYPE,NULLS,UNIQUE
Endkunde_Nr,category,0/1526315,41741
Endkunde,category,0/1526315,39122
Ek_Abc,category,5389/1520926,8
Ek_Boni,category,2408/1523907,3
Ek_Plz,category,475/1525840,3468
Ek_Ort,category,474/1525841,3504
Ek_Land,category,474/1525841,60
Ek_Hb_Apg_Kurzz,category,67241/1459074,95
Ek_Aktiv,category,0/1526315,2
Agentur,category,770366/755949,4606


# Prepare Endkunden Information

In [15]:
def collect(s, sep=', '):
    return sep.join(map(str, s.unique()))

def first(s):
    return s[0]

ek_info = (bd.groupby('Endkunde_Nr')
           .agg({'Endkunde': ['nunique', first]})
           .set_axis(['n', 'name'], axis='columns', inplace=False))

KeyError: 0

In [13]:
qgrid.show_grid(ek_info)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…